In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

In [3]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8744/nasa-engineers-checking-insights-weather-sensors/" target="_self">NASA Engineers Checking InSight's Weather Sensors</a></div>

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA Engineers Checking InSight's Weather Sensors"

In [7]:
# Use the parent element to find the paragraph text.
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p


'An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.'

### Featured Images

In [8]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()


In [10]:
# find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()


In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel


'/spaceimages/images/largesize/PIA17009_hires.jpg'

In [13]:
# add base url to create absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17009_hires.jpg'

In [14]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

 ### Mars Weather

In [16]:

# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)



In [17]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [18]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

<table class="mb_table" id="weather_observation" style="width:100%;">
 <thead>
  <tr>
   <th colspan="2" scope="col">
    Time
   </th>
   <th colspan="3" id="temperature_lbl" scope="col">
    Air Temperature (
    <span class="lbl_fahrenheit">
     °F
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_celsius fadeBlack">
     °C
    </span>
    )
   </th>
   <th colspan="4" id="windspeed_lbl" scope="col">
    Wind Speed (
    <span class="lbl_mph">
     mph
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_mps fadeBlack">
     m/s
    </span>
    )
   </th>
   <th colspan="3" id="pressure_lbl" scope="col">
    Pressure (Pa)
   </th>
  </tr>
 </thead>
 <tbody>
  <tr id="weather_top">
   <th class="sol" scope="row">
    Date
   </th>
   <th class="sol" scope="row">
    Sol
   </th>
   <td class="temperature max">
    Max.
   </td>
   <td class="temperature avg">
    Avg.
   </td>
   <td class="temperature min">
    Min.
   </td>
   <t

### D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles
    Hemispheres

In [110]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

base_url = 'https://astrogeology.usgs.gov'

html = browser.html
hemisphere_soup = soup(html, 'html.parser')
hemisphere_items = hemisphere_soup.find_all('div', class_=['description'])



In [111]:
hemisphere_items

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [161]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
hemi_image_data = {}

# 3. Write code to retrieve the image urls and titles for each hemisphere.
for hemi_item in hemisphere_items:
    #get url for each hemisphere and pull data
    hemi_url =base_url + hemi_item.a['href']
    
    #get title for each hemisphere 
    browser.visit(hemi_url)
    html = browser.html
    hemi_soup = soup(html, 'html.parser')
    hemi_title = hemi_soup.find('h2').text
   
    
    # get image url for each hemisphere
    hemi_image_url = hemi_soup.find('img', class_="wide-image").get('src')
    hemi_image_url = base_url + hemi_image_url
   
    
    hemi_image_data["img_url"] = hemi_image_url
    hemi_image_data["title"]= hemi_title
    hemisphere_image_urls.append(hemi_image_data)
    
    

In [162]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls


[{'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [163]:
# 5. Quit the browser
browser.quit()